In [4]:
%pip install pingouin --upgrade


     ---------------------------------------- 0.0/198.6 kB ? eta -:--:--
     ------------ -------------------------- 61.4/198.6 kB 3.4 MB/s eta 0:00:01
     ------------------ -------------------- 92.2/198.6 kB 1.8 MB/s eta 0:00:01
     ----------------------- -------------- 122.9/198.6 kB 1.0 MB/s eta 0:00:01
     ----------------------- -------------- 122.9/198.6 kB 1.0 MB/s eta 0:00:01
     ----------------------- -------------- 122.9/198.6 kB 1.0 MB/s eta 0:00:01
     ----------------------- -------------- 122.9/198.6 kB 1.0 MB/s eta 0:00:01
     ----------------------- -------------- 122.9/198.6 kB 1.0 MB/s eta 0:00:01
     ----------------------- -------------- 122.9/198.6 kB 1.0 MB/s eta 0:00:01
     ----------------------- -------------- 122.9/198.6 kB 1.0 MB/s eta 0:00:01
     ----------------------- -------------- 122.9/198.6 kB 1.0 MB/s eta 0:00:01
     ----------------------- -------------- 122.9/198.6 kB 1.0 MB/s eta 0:00:01
     ----------------------- -------------- 12

In [1]:
import os 
import sys
import pandas as pd
import numpy as np



parent_dir = os.path.dirname(os.getcwd())
src_dir = os.path.join(parent_dir, 'src')

sys.path.insert(0, src_dir)

import read
#from evaluation import inter_annotator

In [4]:
read.prepare_dataset_for_pearson('age', 'sd', 'young_prob', 'old_prob')

,name,classifier,young_prob,old_prob,type,rating.mean
0,adelaide,ibombSwin,0.322830,0.283650,real,-0.617647
1,adelaide,ibombVit,0.499478,0.062098,real,-0.617647
2,adelaide,nateraw,0.798554,0.010105,real,-0.617647
3,alasdair,ibombSwin,0.549919,0.184914,real,18.709677
4,alasdair,ibombVit,0.644595,0.008137,real,18.709677
...,...,...,...,...,...,...
352,zahara,ibombVit,0.922265,0.023308,madeup,-11.903226
353,zahara,nateraw,0.941309,0.001069,madeup,-11.903226
354,zubaida,ibombSwin,0.127485,0.799051,madeup,9.818182
355,zubaida,ibombVit,0.185573,0.698224,madeup,9.818182


In [1]:
import os
import sys
import pandas as pd 
import pingouin as pg 

parent_dir = os.path.dirname(os.getcwd())
src_dir = os.path.join(parent_dir, 'src')


sys.path.insert(0, src_dir)
import read

gender_df = {'sd': read.prepare_dataset_for_pearson('gender', 'sd', 'female_prob'),
          'vqgan': read.prepare_dataset_for_pearson('gender', 'vqgan', 'female_prob')}

age_df = {'sd': read.prepare_dataset_for_pearson('age', 'sd', 'old_prob'),
          'vqgan': read.prepare_dataset_for_pearson('age', 'vqgan', 'old_prob')}

In [ ]:
def compute_pearson():

    pg.corr(df[])

In [2]:
pg.corr(gender_df['sd']['rating.mean'], gender_df['sd']['female_prob'])

,n,r,CI95%,p-val,BF10,power
pearson,537,0.785928,"[0.75, 0.82]",8.710884e-114,1.045e+110,1.0


In [3]:
test = read.get_merge_dataset('gender', 'sd')

In [4]:
test

,name,rating.mean,type,classifier,image_idx,female_prob,male_prob
0,adelaide,45.727273,real,rizvan,0,0.122037,0.877963
1,adelaide,45.727273,real,rizvan,1,0.975754,0.024246
2,adelaide,45.727273,real,rizvan,2,0.950392,0.049608
3,adelaide,45.727273,real,rizvan,3,0.809022,0.190978
4,adelaide,45.727273,real,rizvan,4,0.967867,0.032133
...,...,...,...,...,...,...,...
10735,zubaida,29.303030,madeup,leilab,15,0.777253,0.222747
10736,zubaida,29.303030,madeup,leilab,16,0.548553,0.451447
10737,zubaida,29.303030,madeup,leilab,17,0.924603,0.075397
10738,zubaida,29.303030,madeup,leilab,18,0.912418,0.087582


In [13]:
test.groupby(['name', 'classifier'], as_index=False).agg({'female_prob': 'mean', 'male_prob': 'mean', 'type': 'first', 'rating.mean': 'first'})

,name,classifier,female_prob,male_prob,type,rating.mean
0,adelaide,leilab,0.826986,0.173014,real,45.727273
1,adelaide,rizvan,0.778883,0.221117,real,45.727273
2,adelaide,rizvan2,0.981421,0.018579,real,45.727273
3,adelina,leilab,0.900916,0.099084,real,47.771429
4,adelina,rizvan,0.973326,0.026674,real,47.771429
...,...,...,...,...,...,...
532,zahara,rizvan,0.949736,0.050264,madeup,41.675676
533,zahara,rizvan2,0.995689,0.004311,madeup,41.675676
534,zubaida,leilab,0.845475,0.154525,madeup,29.303030
535,zubaida,rizvan,0.905110,0.094890,madeup,29.303030


In [ ]:
import pingouin as pg
